In [1]:
import os

if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('../src')

In [2]:
from log_config import logger
from config import load_config, get_abs_path
from preprocess_data import clean_data, convert_json_to_df

import pandas as pd
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))

config = load_config()

paths = config['paths']

[nltk_data] Downloading package stopwords to /home/felip/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/felip/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# df_applicants
path_applicants = get_abs_path(paths['applicants_json'])
cols_applicants = [
    'infos_basicas',
    'informacoes_pessoais',
    'informacoes_profissionais',
    'formacao_e_idiomas',
]
df_applicants = convert_json_to_df(
    path=path_applicants, index_col='cod_applicant', cols_normalize=cols_applicants
)
logger.info('df_applicants %s', df_applicants.shape)

INFO:2025-04-21 19:30:37,171:root:df_applicants (42482, 46)


In [4]:
# df_prospects
path_prospects = get_abs_path(paths['prospects_json'])
df_prospects = convert_json_to_df(
    path=path_prospects, index_col='cod_vaga', explode_col='prospects'
)
logger.info('df_prospects %s', df_prospects.shape)

INFO:2025-04-21 19:30:39,356:root:df_prospects (56702, 10)


In [5]:
# df_vagas
path_vagas = get_abs_path(paths['vagas_json'])
cols_vagas = ['informacoes_basicas', 'perfil_vaga', 'beneficios']
df_vagas = convert_json_to_df(
    path=path_vagas, index_col='cod_vaga', cols_normalize=cols_vagas
)
logger.info('df_vagas %s', df_vagas.shape)

INFO:2025-04-21 19:30:42,343:root:df_vagas (14081, 45)


In [6]:
logger.info('# --- TRATANDO COLUNAS TABELA APPLICANTS ---')

colunas_texto_applicants = [
    'cv_pt',
    'objetivo_profissional',
    'fonte_indicacao',
    'titulo_profissional',
    'area_atuacao',
    'nivel_academico',
    'cursos',
]
colunas_datas_applicants = ['data_criacao', 'data_atualizacao', 'data_nascimento']
df_applicants = clean_data(
    df_applicants,
    colunas_texto_applicants,
    colunas_datas_applicants,
    ['ano_conclusao'],
    ['remuneracao'],
)

INFO:2025-04-21 19:30:42,356:root:# --- TRATANDO COLUNAS TABELA APPLICANTS ---


In [7]:
logger.info('# --- TRATANDO COLUNAS TABELA VAGAS ---  ')
colunas_texto_vagas = [
    'titulo_vaga',
    'tipo_contratacao',
    'nivel_academico',
    'areas_atuacao',
    'principais_atividades',
    'competencia_tecnicas_e_comportamentais',
    'demais_observacoes',
    'equipamentos_necessarios',
    'habilidades_comportamentais_necessarias',
]
colunas_datas_vagas = [
    'limite_esperado_para_contratacao',
    'data_inicial',
    'data_final',
]

df_vagas = clean_data(df_vagas, colunas_texto_vagas, colunas_datas_vagas)

INFO:2025-04-21 19:31:11,399:root:# --- TRATANDO COLUNAS TABELA VAGAS ---  


In [8]:
logger.info('# --- TRATANDO COLUNAS TABELA PROSPECTS ---')

colunas_texto_prospects = ['titulo', 'situacao_candidado', 'comentario']
df_prospects = clean_data(df_prospects, colunas_texto_prospects)
logger.info('Data preprocessing completed.')

INFO:2025-04-21 19:31:15,668:root:# --- TRATANDO COLUNAS TABELA PROSPECTS ---
INFO:2025-04-21 19:31:16,420:root:Data preprocessing completed.


## Consolidar dados na camada Gold

In [9]:
# # Merge prospects + applicants (via nome)
# df = df_prospects.merge(df_applicants, on="nome", how="left", suffixes=("", "_app"))
# df

In [10]:
df_prospects

,cod_vaga,titulo,nome,codigo,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador
0,4530,consultor control m,José Vieira,25632,encaminhado requisitante,25-03-2021,25-03-2021,encaminhado pj r 7200 hora,Ana Lívia Moreira
1,4530,consultor control m,Srta. Isabela Cavalcante,25529,encaminhado requisitante,22-03-2021,23-03-2021,encaminhado r 600000 clt full nao empresa aberta,Ana Lívia Moreira
2,4531,20212607395peoplesoft application enginedomain...,Sra. Yasmin Fernandes,25364,contratado decision,17-03-2021,12-04-2021,data inicio 12 04 2021,Juliana Cassiano
3,4531,20212607395peoplesoft application enginedomain...,Alexia Barbosa,25360,encaminhado requisitante,17-03-2021,17-03-2021,None,Juliana Cassiano
4,4532,None,NaN,NaN,None,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...
56697,14219,None,NaN,NaN,None,NaN,NaN,None,NaN
56698,14220,consultor senior especialista sap lestra 1433,Ana Cardoso,16828,desistiu,26-02-2025,28-02-2025,recebeu confirmacao outro processo seletivo re...,Elisa Nunes
56699,14220,consultor senior especialista sap lestra 1433,Pedro Lucas das Neves,15042,encaminhado requisitante,28-02-2025,28-02-2025,None,Elisa Nunes
56700,14221,consultor senior oracle epm fccs 1434,Maria Eduarda Cassiano,49190,prospect,26-02-2025,26-02-2025,None,Luna Correia


In [11]:
# Merge com vagas (via cod_vaga)
df = df_prospects.merge(df_vagas, on='cod_vaga', how='left', suffixes=('', '_vaga'))

In [12]:
df

,cod_vaga,titulo,nome,codigo,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador,data_requicisao,...,nivel_ingles,nivel_espanhol,areas_atuacao,principais_atividades,competencia_tecnicas_e_comportamentais,demais_observacoes,equipamentos_necessarios,habilidades_comportamentais_necessarias,valor_venda,valor_compra_1
0,4530,consultor control m,José Vieira,25632,encaminhado requisitante,25-03-2021,25-03-2021,encaminhado pj r 7200 hora,Ana Lívia Moreira,10-03-2021,...,Nenhum,Nenhum,ti desenvolvimento programacao,experiencia comprovada projetos controlm,experiencia comprovada projetos controlm,contratacao pj projeto pontual 2 3 meses clien...,nenhum,None,-,R$
1,4530,consultor control m,Srta. Isabela Cavalcante,25529,encaminhado requisitante,22-03-2021,23-03-2021,encaminhado r 600000 clt full nao empresa aberta,Ana Lívia Moreira,10-03-2021,...,Nenhum,Nenhum,ti desenvolvimento programacao,experiencia comprovada projetos controlm,experiencia comprovada projetos controlm,contratacao pj projeto pontual 2 3 meses clien...,nenhum,None,-,R$
2,4531,20212607395peoplesoft application enginedomain...,Sra. Yasmin Fernandes,25364,contratado decision,17-03-2021,12-04-2021,data inicio 12 04 2021,Juliana Cassiano,10-03-2021,...,Nenhum,Nenhum,gestao alocacao recursos ti,key skills required the job are peoplesoft app...,recurso peoplesoft responsabilidades projetar ...,remoto presencial tempo indeterminado,notebook padrao,None,-,hora
3,4531,20212607395peoplesoft application enginedomain...,Alexia Barbosa,25360,encaminhado requisitante,17-03-2021,17-03-2021,None,Juliana Cassiano,10-03-2021,...,Nenhum,Nenhum,gestao alocacao recursos ti,key skills required the job are peoplesoft app...,recurso peoplesoft responsabilidades projetar ...,remoto presencial tempo indeterminado,notebook padrao,None,-,hora
4,4532,None,NaN,NaN,None,NaN,NaN,None,NaN,11-03-2021,...,Nenhum,Nenhum,gestao alocacao recursos ti,consultor senior experiencia localizacao brasi...,desejavel conhecimento segmento ol gas atendim...,apenas pj clt full,None,tempo projeto 1 ano requested start date110320...,-,Valor Aberto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56697,14219,None,NaN,NaN,None,NaN,NaN,None,NaN,26-02-2025,...,Básico,<NA>,ti projetos,importante idioma ingles espanhol level 9 8 le...,skills aspnet javascript react csharp applicat...,primary contact pfigueiredo wbs z026gl70 indir...,None,None,168 - p/ hora,Fechado
56698,14220,consultor senior especialista sap lestra 1433,Ana Cardoso,16828,desistiu,26-02-2025,28-02-2025,recebeu confirmacao outro processo seletivo re...,Elisa Nunes,26-02-2025,...,Intermediário,<NA>,ti sap,vaga consultor senior especialista sap lestra ...,conhecimentos tecnicos requeridos consultor sa...,None,None,None,168 -,Fechado
56699,14220,consultor senior especialista sap lestra 1433,Pedro Lucas das Neves,15042,encaminhado requisitante,28-02-2025,28-02-2025,None,Elisa Nunes,26-02-2025,...,Intermediário,<NA>,ti sap,vaga consultor senior especialista sap lestra ...,conhecimentos tecnicos requeridos consultor sa...,None,None,None,168 -,Fechado
56700,14221,consultor senior oracle epm fccs 1434,Maria Eduarda Cassiano,49190,prospect,26-02-2025,26-02-2025,None,Luna Correia,26-02-2025,...,Básico,<NA>,ti projetos,vaga consultor senior oracle epm fccs modelo t...,conhecimentos tecnicos requeridos oracle epm f...,None,None,None,168 -,Fechado


In [13]:
df.isnull().mean().sort_values(ascending=False)

regiao                                     0.909580
equipamentos_necessarios                   0.783588
data_final                                 0.767574
data_inicial                               0.767116
habilidades_comportamentais_necessarias    0.760837
comentario                                 0.743254
limite_esperado_para_contratacao           0.381186
prazo_contratacao                          0.258104
prioridade_vaga                            0.244965
demais_observacoes                         0.239039
ultima_atualizacao                         0.120913
nivel_espanhol                             0.091196
data_candidatura                           0.051903
nome                                       0.051903
recrutador                                 0.051903
titulo                                     0.051903
situacao_candidado                         0.051903
codigo                                     0.051903
tipo_contratacao                           0.037353
cidade      

In [14]:
missing = df.isnull().mean().sort_values(ascending=False)
print('Colunas com maior proporção de valores ausentes:')
print(missing[missing > 0.3])

# precisa melhorar o preenchimento de valores ausentes
colunas_cat = df.select_dtypes(include='object').columns
for col in colunas_cat:
    df[col] = df[col].fillna('desconhecido')

# Precisa melhorar a padronização de datas
df['data_candidatura'] = pd.to_datetime(
    df['data_candidatura'], errors='coerce', dayfirst=True
)

output_path = get_abs_path(paths['dataset_consolidado'])
df.to_parquet(output_path, index=False)
print(f'Dataset consolidado salvo em: {output_path}')

Colunas com maior proporção de valores ausentes:
regiao                                     0.909580
equipamentos_necessarios                   0.783588
data_final                                 0.767574
data_inicial                               0.767116
habilidades_comportamentais_necessarias    0.760837
comentario                                 0.743254
limite_esperado_para_contratacao           0.381186
dtype: float64
Dataset consolidado salvo em: /home/felip/projetos/datathon-mle/Datathon Decision/4_gold/dataset_consolidado.parquet


## Definir target

In [15]:
situacao_candidado_nao_aprovado = [
    'nao aprovado rh',
    'nao aprovado cliente',
    'nao aprovado requisitante',
    'recusado',
]
situacao_candidado_aprovado = [
    'contratado decision',
    'prospect',
    'entrevista tecnica',
    'proposta aceita',
    'contratado hunting',
    'entrevista cliente',
    'documentacao clt',
    'documentacao pj',
    'documentacao cooperado',
    'encaminhar proposta',
]
situacao_candidado_desistente = ['desistiu', 'desistiu contratacao', 'avaliacao rh']
situacao_candidato_inicial = [
    'encaminhado requisitante',
    'interesse nesta vaga',
    'inscrito',
]

mapping = {}
for s in situacao_candidado_nao_aprovado:
    mapping[s] = 'nao_aprovado'
for s in situacao_candidado_aprovado:
    mapping[s] = 'aprovado'
for s in situacao_candidado_desistente:
    mapping[s] = 'desistente'
for s in situacao_candidato_inicial:
    mapping[s] = 'inicial'

In [16]:
df['situacao_candidado'] = df['situacao_candidado'].replace(mapping)

# Definindo a target
df['target'] = (df['situacao_candidado'] == 'aprovado').astype(int)

# Remover colunas que não podem ser usadas como preditoras
colunas_remover = [
    'analista_responsavel',
    'cidade',
    'cliente',
    'cod_vaga',
    'codigo',
    'data_candidatura',
    'data_final',
    'data_inicial',
    'data_requicisao',
    'empresa_divisao',
    'estado',
    'limite_esperado_para_contratacao',
    'local_trabalho',
    'nome',
    'recrutador',
    'regiao',
    'requisitante',
    'situacao_candidado',
    'solicitante_cliente',
    'ultima_atualizacao',
]

# colunas_features = ["titulo", "comentario", "titulo_vaga", "tipo_contratacao", "prazo_contratacao", "prioridade_vaga",
#                     "nivel profissional", "nivel_academico", "nivel_ingles", "nivel_espanhol", "areas_atuacao",
#                     "principais_atividades", "competencia_tecnicas_e_comportamentais", "demais_observacoes", "equipamentos_necessarios",
#                     "habilidades_comportamentais_necessarias", "valor_venda", "valor_compra_1" ]

df_model = df.drop(columns=[col for col in colunas_remover if col in df.columns])
# Salva o dataset de modelagem usando caminho absoluto
df_model.to_parquet(get_abs_path(paths['dataset_modelagem']), index=False)

## Feature Engineering

In [17]:
config = load_config()
paths = config['paths']
fe_cfg = config['feature_engineering']
for k in paths:
    paths[k] = get_abs_path(paths[k])
stop_words = set(stopwords.words('portuguese'))
palavras_chave = fe_cfg['palavras_chave']
n_clusters = fe_cfg['n_clusters']

In [18]:
df = df_model.copy()

In [19]:
def coluna_valida(df: pd.DataFrame, col: str) -> bool:
    """Check if a column is valid for feature creation."""
    if col not in df.columns:
        return False
    missing = df[col].isnull().mean()
    if missing > 0.5:
        logger.info(
            f'[SKIP] Coluna "{col}" com {missing:.0%} missing, feature não criada.'
        )
        return False
    vc = df[col].value_counts(normalize=True, dropna=True)
    if not vc.empty and vc.iloc[0] > 0.9:
        logger.info(
            f'[SKIP] Coluna "{col}" com valor dominante ({vc.index[0]}) em {vc.iloc[0]:.0%}, feature não criada.'
        )
        return False
    return True

In [20]:
if coluna_valida(df, 'nivel_academico'):
    df = pd.get_dummies(
        df, columns=['nivel_academico'], prefix='nivel_acad', dummy_na=True
    )
    logger.info('[OK] One-hot de nivel_academico criado.')
else:
    logger.info('[SKIP] One-hot de nivel_academico não criado.')

INFO:2025-04-21 19:31:52,932:root:[OK] One-hot de nivel_academico criado.


In [21]:
if coluna_valida(df, 'tipo_contratacao'):
    df = pd.get_dummies(
        df, columns=['tipo_contratacao'], prefix='tipo_contr', dummy_na=True
    )
    logger.info('[OK] One-hot de tipo_contratacao criado.')
else:
    logger.info('[SKIP] One-hot de tipo_contratacao não criado.')

INFO:2025-04-21 19:31:52,998:root:[OK] One-hot de tipo_contratacao criado.


In [22]:
df_model.columns

Index(['titulo', 'comentario', 'titulo_vaga', 'tipo_contratacao',
       'prazo_contratacao', 'prioridade_vaga', 'nivel profissional',
       'nivel_academico', 'nivel_ingles', 'nivel_espanhol', 'areas_atuacao',
       'principais_atividades', 'competencia_tecnicas_e_comportamentais',
       'demais_observacoes', 'equipamentos_necessarios',
       'habilidades_comportamentais_necessarias', 'valor_venda',
       'valor_compra_1', 'target'],
      dtype='object')

In [23]:
from transformers import pipeline
from typing import Any

pipe = pipeline(
    'text-classification', model='tabularisai/multilingual-sentiment-analysis'
)


def tamanho_texto(texto: Any) -> int:
    if pd.isnull(texto):
        return 0
    return len(str(texto))


def n_palavras(texto: Any) -> int:
    if pd.isnull(texto):
        return 0
    return len(str(texto).split())


def classifica_sentimento_do_texto(texto: Any):
    if pd.isnull(texto):
        return 0
    return pipe(texto)


campos_texto = [
    'principais_atividades',
    'competencia_tecnicas_e_comportamentais',
    'demais_observacoes',
    'comentario',
]
for campo in campos_texto:
    if campo in df.columns:
        df[f'{campo}_nchar'] = df[campo].apply(tamanho_texto)
        df[f'{campo}_nwords'] = df[campo].apply(n_palavras)
        # df[f"{campo}_sentiment"] = pd.DataFrame(pipe(df[campo].to_list()))['label']
        logger.info(f'[OK] Features de texto para {campo} criadas.')

Device set to use cuda:0
INFO:2025-04-21 19:31:24,072:root:[OK] Features de texto para principais_atividades criadas.
INFO:2025-04-21 19:31:24,287:root:[OK] Features de texto para competencia_tecnicas_e_comportamentais criadas.
INFO:2025-04-21 19:31:24,390:root:[OK] Features de texto para demais_observacoes criadas.
INFO:2025-04-21 19:31:24,483:root:[OK] Features de texto para comentario criadas.


In [24]:
import pandas as pd
import logger
from typing import Any, List
from sentence_transformers import SentenceTransformer


class TextFeatureGenerator:
    def __init__(self):
        logger.info('[Init] Carregando modelo de embeddings...')
        self.embedding_model = SentenceTransformer(
            'paraphrase-multilingual-MiniLM-L12-v2'
        )

    def tamanho_texto(self, texto: Any) -> int:
        if pd.isnull(texto):
            return 0
        return len(str(texto))

    def n_palavras(self, texto: Any) -> int:
        if pd.isnull(texto):
            return 0
        return len(str(texto).split())

    def gerar_embeddings_agregados(
        self, textos: List[str], batch_size: int = 128
    ) -> pd.DataFrame:
        logger.info(f'[Embeddings] Iniciando geração para {len(textos)} textos...')

        embeddings = self.embedding_model.encode(
            textos,
            batch_size=batch_size,
            show_progress_bar=True,
            convert_to_numpy=True,
            normalize_embeddings=True,
        )

        logger.info('[Embeddings] Geração finalizada. Criando features agregadas...')
        df_emb = pd.DataFrame(
            {
                'emb_mean': embeddings.mean(axis=1),
                'emb_std': embeddings.std(axis=1),
                'emb_min': embeddings.min(axis=1),
                'emb_max': embeddings.max(axis=1),
            }
        )

        return df_emb

    def transform(self, df: pd.DataFrame, campos_texto: List[str]) -> pd.DataFrame:
        for campo in campos_texto:
            if campo in df.columns:
                logger.info(f'[{campo}] Criando features de tamanho e palavras...')
                # df[f"{campo}_nchar"] = df[campo].apply(self.tamanho_texto)
                # df[f"{campo}_nwords"] = df[campo].apply(self.n_palavras)

                logger.info(f'[{campo}] Criando embeddings agregados...')
                textos = df[campo].fillna('').astype(str).tolist()
                df_emb = self.gerar_embeddings_agregados(textos)
                df_emb.columns = [f'{campo}_{col}' for col in df_emb.columns]

                df = pd.concat([df, df_emb], axis=1)
                logger.info(f'[{campo}] Features de embeddings agregados adicionadas.')

        return df

In [25]:
campos_texto = [
    'principais_atividades',
    'competencia_tecnicas_e_comportamentais',
    'demais_observacoes',
    'comentario',
]

feature_generator = TextFeatureGenerator()
df = feature_generator.transform(df, campos_texto)

INFO:2025-04-21 19:31:56,225:root:[Init] Carregando modelo de embeddings...
INFO:2025-04-21 19:31:56,230:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda:0
INFO:2025-04-21 19:31:56,231:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: paraphrase-multilingual-MiniLM-L12-v2
INFO:2025-04-21 19:31:59,525:root:[principais_atividades] Criando features de tamanho e palavras...
INFO:2025-04-21 19:31:59,526:root:[principais_atividades] Criando embeddings agregados...
INFO:2025-04-21 19:31:59,538:root:[Embeddings] Iniciando geração para 56702 textos...


Batches:   0%|          | 0/443 [00:00<?, ?it/s]

INFO:2025-04-21 19:34:17,698:root:[Embeddings] Geração finalizada. Criando features agregadas...
INFO:2025-04-21 19:34:17,839:root:[principais_atividades] Features de embeddings agregados adicionadas.
INFO:2025-04-21 19:34:17,840:root:[competencia_tecnicas_e_comportamentais] Criando features de tamanho e palavras...
INFO:2025-04-21 19:34:17,841:root:[competencia_tecnicas_e_comportamentais] Criando embeddings agregados...
INFO:2025-04-21 19:34:17,849:root:[Embeddings] Iniciando geração para 56702 textos...


Batches:   0%|          | 0/443 [00:00<?, ?it/s]

INFO:2025-04-21 19:36:26,043:root:[Embeddings] Geração finalizada. Criando features agregadas...
INFO:2025-04-21 19:36:26,185:root:[competencia_tecnicas_e_comportamentais] Features de embeddings agregados adicionadas.
INFO:2025-04-21 19:36:26,187:root:[demais_observacoes] Criando features de tamanho e palavras...
INFO:2025-04-21 19:36:26,188:root:[demais_observacoes] Criando embeddings agregados...
INFO:2025-04-21 19:36:26,202:root:[Embeddings] Iniciando geração para 56702 textos...


Batches:   0%|          | 0/443 [00:00<?, ?it/s]

INFO:2025-04-21 19:37:28,966:root:[Embeddings] Geração finalizada. Criando features agregadas...
INFO:2025-04-21 19:37:29,120:root:[demais_observacoes] Features de embeddings agregados adicionadas.
INFO:2025-04-21 19:37:29,123:root:[comentario] Criando features de tamanho e palavras...
INFO:2025-04-21 19:37:29,125:root:[comentario] Criando embeddings agregados...
INFO:2025-04-21 19:37:29,140:root:[Embeddings] Iniciando geração para 56702 textos...


Batches:   0%|          | 0/443 [00:00<?, ?it/s]

INFO:2025-04-21 19:38:01,913:root:[Embeddings] Geração finalizada. Criando features agregadas...
INFO:2025-04-21 19:38:02,059:root:[comentario] Features de embeddings agregados adicionadas.


In [ ]:
from tqdm import tqdm
from sentence_transformers import util
from rapidfuzz import fuzz

embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')


def gerar_embeddings(textos: pd.Series):
    textos = textos.fillna('').astype(str).tolist()
    return embedding_model.encode(
        textos, convert_to_tensor=True, normalize_embeddings=True
    )


def similaridade_string(t1, t2):
    if pd.isnull(t1) or pd.isnull(t2):
        return 0
    return fuzz.token_sort_ratio(str(t1), str(t2)) / 100


def adicionar_similaridade_titulo_vaga(
    df: pd.DataFrame, col1='titulo', col2='titulo_vaga'
) -> pd.DataFrame:
    if col1 in df.columns and col2 in df.columns:
        tqdm.pandas(desc='[String Similarity]')
        df['titulo_sim_ratio'] = df.progress_apply(
            lambda row: similaridade_string(row[col1], row[col2]), axis=1
        )

        # Similaridade semântica com embeddings
        tqdm.write('[Embeddings] Gerando embeddings dos títulos...')
        emb1 = gerar_embeddings(df[col1].fillna('').astype(str))
        emb2 = gerar_embeddings(df[col2].fillna('').astype(str))

        tqdm.write('[Embeddings] Calculando similaridade de cosseno...')
        similarities = util.cos_sim(emb1, emb2).diagonal().cpu().numpy()
        df['sim_titulo_vs_vaga'] = similarities

    return df


df = adicionar_similaridade_titulo_vaga(df)

[String Similarity]: 100%|██████████| 56702/56702 [00:00<00:00, 103612.11it/s]


[Embeddings] Gerando embeddings dos títulos...


Batches:   0%|          | 0/1772 [00:00<?, ?it/s]

Batches:   0%|          | 0/1772 [00:00<?, ?it/s]

[Embeddings] Calculando similaridade de cosseno...


In [36]:
df_model = df.drop(
    columns=[
        'titulo',
        'comentario',
        'titulo_vaga',
        'prazo_contratacao',
        'prioridade_vaga',
        'nivel profissional',
        'nivel_ingles',
        'nivel_espanhol',
        'areas_atuacao',
        'principais_atividades',
        'competencia_tecnicas_e_comportamentais',
        'demais_observacoes',
        'equipamentos_necessarios',
        'habilidades_comportamentais_necessarias',
        'valor_venda',
        'valor_compra_1',
    ]
)

In [37]:
df_model

,target,nivel_acad_desconhecido,nivel_acad_doutorado completo,nivel_acad_doutorado cursando,nivel_acad_ensino fundamental completo,nivel_acad_ensino medio completo,nivel_acad_ensino medio incompleto,nivel_acad_ensino superior completo,nivel_acad_ensino superior cursando,nivel_acad_ensino superior incompleto,...,demais_observacoes_emb_mean,demais_observacoes_emb_std,demais_observacoes_emb_min,demais_observacoes_emb_max,comentario_emb_mean,comentario_emb_std,comentario_emb_min,comentario_emb_max,titulo_sim_ratio,sim_titulo_vs_vaga
0,0,False,False,False,False,False,False,True,False,False,...,-0.000089,0.051031,-0.161850,0.117181,-0.000235,0.051030,-0.176741,0.204723,1.000000,1.000000
1,0,False,False,False,False,False,False,True,False,False,...,-0.000089,0.051031,-0.161850,0.117181,0.000284,0.051030,-0.155262,0.165039,1.000000,1.000000
2,1,False,False,False,False,True,False,False,False,False,...,0.000239,0.051030,-0.185271,0.210202,0.000357,0.051030,-0.165430,0.155121,1.000000,1.000000
3,0,False,False,False,False,True,False,False,False,False,...,0.000239,0.051030,-0.185271,0.210202,0.000110,0.051031,-0.177052,0.497576,1.000000,1.000000
4,0,False,False,False,False,False,False,False,False,False,...,0.000033,0.051031,-0.153048,0.165421,0.000110,0.051031,-0.177052,0.497576,0.066667,0.188191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56697,0,False,False,False,False,False,False,True,False,False,...,0.000245,0.051030,-0.129788,0.161187,0.000110,0.051031,-0.177052,0.497576,0.250000,0.124156
56698,0,False,False,False,False,False,False,True,False,False,...,0.000110,0.051031,-0.177052,0.497576,0.000314,0.051030,-0.126359,0.117563,1.000000,1.000000
56699,0,False,False,False,False,False,False,True,False,False,...,0.000110,0.051031,-0.177052,0.497576,0.000110,0.051031,-0.177052,0.497576,1.000000,1.000000
56700,1,False,False,False,False,False,False,True,False,False,...,0.000110,0.051031,-0.177052,0.497576,0.000110,0.051031,-0.177052,0.497576,1.000000,1.000000


In [38]:
df_model.to_parquet(paths['dataset_features'], index=False)

## Treinamento

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report,
)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

model_cfg = config['model']

if 'target' in df.columns:
    y = df['target']
else:
    raise ValueError(
        'Coluna target não encontrada no dataset de features. Execute a etapa de definição do target.'
    )

# Estamos pegando todas as features numéricas por enquanto, vamos melhorar isso depois removendo as de menor relevância (exceto target)
features = [
    col
    for col in df.columns
    if col != 'target' and pd.api.types.is_numeric_dtype(df[col])
]
X = df[features]
print(f'Features usadas no treino: {features}')

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=model_cfg['test_size'],
    random_state=model_cfg['random_state'],
    stratify=y,
)

imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()
X_train_imp = imputer.fit_transform(X_train)
X_test_imp = imputer.transform(X_test)
X_train_scaled = scaler.fit_transform(X_train_imp)
X_test_scaled = scaler.transform(X_test_imp)

Features usadas no treino: ['nivel_acad_desconhecido', 'nivel_acad_doutorado completo', 'nivel_acad_doutorado cursando', 'nivel_acad_ensino fundamental completo', 'nivel_acad_ensino medio completo', 'nivel_acad_ensino medio incompleto', 'nivel_acad_ensino superior completo', 'nivel_acad_ensino superior cursando', 'nivel_acad_ensino superior incompleto', 'nivel_acad_ensino tecnico completo', 'nivel_acad_ensino tecnico cursando', 'nivel_acad_ensino tecnico incompleto', 'nivel_acad_mestrado completo', 'nivel_acad_mestrado cursando', 'nivel_acad_pos graduacao completo', 'nivel_acad_pos graduacao cursando', 'nivel_acad_pos graduacao incompleto', 'nivel_acad_nan', 'tipo_contr_candidato podera escolher hunting pj autonomo', 'tipo_contr_candidato podera escolher pj autonomo', 'tipo_contr_clt cotas', 'tipo_contr_clt cotas clt full', 'tipo_contr_clt cotas cooperado', 'tipo_contr_clt cotas cooperado estagiario pj autonomo', 'tipo_contr_clt cotas cooperado estagiario pj autonomo hunting', 'tipo_co

In [42]:
models = {
    'RandomForest': RandomForestClassifier(
        n_estimators=model_cfg['rf_n_estimators'],
        random_state=model_cfg['random_state'],
    ),
    'LogisticRegression': LogisticRegression(
        max_iter=model_cfg['lr_max_iter'], random_state=model_cfg['random_state']
    ),
}
results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    y_proba = (
        model.predict_proba(X_test_scaled)[:, 1]
        if hasattr(model, 'predict_proba')
        else y_pred
    )
    auc = roc_auc_score(y_test, y_proba)
    f1 = f1_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    results[name] = {'auc': auc, 'f1': f1, 'precision': prec, 'recall': rec}
    print(f'\nModelo: {name}')
    print(f'AUC: {auc:.3f} | F1: {f1:.3f} | Precision: {prec:.3f} | Recall: {rec:.3f}')
    print(classification_report(y_test, y_pred))

best_model_name = max(results, key=lambda k: results[k]['auc'])
best_model = models[best_model_name]


Modelo: RandomForest
AUC: 0.867 | F1: 0.759 | Precision: 0.759 | Recall: 0.760
              precision    recall  f1-score   support

           0       0.82      0.82      0.82      6486
           1       0.76      0.76      0.76      4855

    accuracy                           0.79     11341
   macro avg       0.79      0.79      0.79     11341
weighted avg       0.79      0.79      0.79     11341


Modelo: LogisticRegression
AUC: 0.771 | F1: 0.712 | Precision: 0.607 | Recall: 0.860
              precision    recall  f1-score   support

           0       0.85      0.58      0.69      6486
           1       0.61      0.86      0.71      4855

    accuracy                           0.70     11341
   macro avg       0.73      0.72      0.70     11341
weighted avg       0.74      0.70      0.70     11341



In [44]:
import pickle

with open(paths['modelo_treinado'], 'wb') as f:
    pickle.dump(
        {
            'model': best_model,
            'imputer': imputer,
            'scaler': scaler,
            'features': features,
        },
        f,
    )
print(
    f'\nMelhor modelo salvo: {best_model_name} (AUC={results[best_model_name]["auc"]:.3f})'
)
print(f'Arquivo: {paths["modelo_treinado"]}')


Melhor modelo salvo: RandomForest (AUC=0.867)
Arquivo: /home/felip/projetos/datathon-mle/Datathon Decision/4_gold/modelo_treinado.pkl


In [47]:
# Verificando importância das features. Vamos utilizar isso para filtrar as features posteriormente
importances = models['RandomForest'].feature_importances_
feature_importance = pd.Series(importances, index=features).sort_values(ascending=False)
print('\nImportância das features (RandomForest):')
print(feature_importance.head(20))
feature_importance.to_csv(paths['feature_importance_rf'])

coefs = models['LogisticRegression'].coef_[0]
coef_importance = pd.Series(coefs, index=features).sort_values(key=abs, ascending=False)
print('\nCoeficientes das features (LogisticRegression):')
print(coef_importance.head(20))
coef_importance.to_csv(paths['feature_importance_lr'])


Importância das features (RandomForest):
sim_titulo_vs_vaga                                 0.103738
comentario_emb_max                                 0.059939
comentario_nwords                                  0.047920
comentario_emb_min                                 0.043927
comentario_emb_mean                                0.040016
comentario_nchar                                   0.037879
principais_atividades_emb_max                      0.035895
competencia_tecnicas_e_comportamentais_emb_max     0.035683
principais_atividades_emb_min                      0.035062
competencia_tecnicas_e_comportamentais_emb_mean    0.034952
principais_atividades_nchar                        0.034588
competencia_tecnicas_e_comportamentais_emb_min     0.034432
principais_atividades_emb_mean                     0.034355
competencia_tecnicas_e_comportamentais_nchar       0.033693
principais_atividades_nwords                       0.031707
demais_observacoes_emb_mean                        0.03070